### Convolutional Neural Networks
In house price prediction, with a one hidden layer architecture, the networks learns between $\sqrt{n}$ and $n$ combinations of features. In images, as opposed to say in house price prediction, the most interesting combination of features (pixels) come from pixels that are close to each other, than some random selection of pixels, in here, the order of features matter, as opposed to the problem in house price prediction.

For image data, we should have an order of magnitude more combinations of features, and each of these combinations should only be combinations of features from a small rectangle patch in the image. With $n$ input pixels for an input image, each patch goes through a network separately, so the network, in total should learn between $n$ and $n^2$ combinations of feature. Having a network learn combinations of all of the input features, pixels, is inefficient, as it ignores the insight above.

In computer vision it is known that if you multiply a certain kernel, here we call it $W$, associated with certain visual patterns, like edges, with certain section of the image, the output of that dot product gives the result of whether there is edges at that location of the image or not, now if we slide this $W$ matrice across the image, taking the dot product, we end up with a new image, called "feature map", showing where the pattern defined by $W$ was present at the input image. This is at the core of "Convolutional Neural Networks", CNNs.

Take this simple example:

$$
\begin{align}
W &= 
\begin{bmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33}
\end{bmatrix} \\
X &= 
\begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14} & x_{15} \\
x_{21} & x_{22} & x_{23} & x_{24} & x_{25} \\
x_{31} & x_{32} & x_{33} & x_{34} & x_{35} \\
x_{41} & x_{42} & x_{43} & x_{44} & x_{45} \\
x_{51} & x_{52} & x_{53} & x_{54} & x_{55}
\end{bmatrix}
\end{align}
$$

The computed "feature" from a local patch of the image, say centered at element $x_{33}$ will be:

$$
\begin{align}
W \cdot X |_{x_{33}} &= w_{11}x_{22} + w_{12}x_{32} + w_{13}w_{42} \\
                     &+ w_{21}x_{23} + w_{22}x_{33} + w_{23}x_{43} \\
                     &+ w_{31}x_{24} + w_{32}x_{34} + w_{33}x_{44}
\end{align}
$$

This value will be treated like any other computed feature from a usuall network, it may have a bias added to it, then probably fed through an activation function, which will represent a neuron or a learned feature, that will get passed to other layers of the network, this way we can define fetures that are functions of small patches of the image. Now if we slide $W$ over the image, taking the dot product $W$ with the pixels centered at each location, we end up with a new image of the same size (depending on edge handling), this is a feature map showing the locations where the "pattern" defined by $W$ is existant in each location. This operation is called *Convolution*.

#### Multichannel Convolutions

For an input image with $n$ pixels, the convolution operation above will create $n$ output features, one for each location in the image. But we can deduce more features from an image by defining multiple weights which define different "patterns" in the image, in a convolutional layer we create $f$ sets of $n$ features, each with a randomly initialized weights, whose detection at each location in the input image will be captured in the feature map. These $f$ feature maps will be created via $f$ convolution operations. Each of these $f$ feature maps is called a *channel* of the layer. Each of the $W_i$ weights associated with a particular feature map are called the convolutional *filter* or a *kernel*.

#### Deep Convolutional Neural Networks

How can we perform the multichannel convolution on an input with multiple channels, for example in a network two convolutional layers? In the first layer of a network with fully connected layers, we have $h_1$ features that are combination of the original input features. In the second layer we have we have $h_2$ combination of previous features, these are features of features, to go from first to second layer we use a $h_1 \times h_2$ weights, each of the $h_2$ features is a function of each of the $h_1$ features in the previous layer. Same intuition applies in convolutional networks also. 

We first transform the input image into $m_1$ feature maps using $m_1$ convolutional filters, the ouput of this first convolutional layer represents if each of the $m_1$ visual patters represented by the weights of each of the $m_1$ filters is present at each location of the image. The next layer of a CNN could contain like the MLP, $m_2$ filters which intutively represents patterns of patterns, and whether they are present at that location of the image. A given location in the image on one of the $m_2$ feature maps is a linear combination of convolving $m_1$ different filters over that same location in each of the corresponding $m_1$ feature maps form the prior layer.

#### Flatten Layer

#### Pooling Layer

#### Forward pass

*Padding*: To avoid shrinking the ouput as a result of convolution, we pad the input around the edges with zeros, 
so that the output will be the same shape as input, in general: 
`P = ((S-1)*W-S+F)/2`, with `F = filter size`, `S = stride`, `W = input size` 
for our case with a stride of 1: `F-1//2`

*Stride*: As pooling layers downsample the image, they reduce the amount of information by a factor of 4, 
so that an image with just half the resolution fed through the network, these pooling layers have been deprecated  in favor of modifying the stride of the convolution operator. The stride is the amount by which the filter is incrementally slid over the image. For us it was 1. With a stride of 2, the filter would be convolved with every other element of the input image, the output will be half the size of input. This is the same we would get with a size 2 pooling, and thus the same reduction in computation, but without as much loss of information. With pooling of size 2, only one-fourth of the elements in the input have any effect on the output, whereas with a stride of 2, every element of the input has some effect on the output. The use of a stride of greater than 1 is thus significantly more prevalent than pooling for downsampling even in the most advanced CNN architectures of today.

In [1]:
import numpy as np

In [2]:
# 1D convolution operator example
input_1d = np.array([1, 2, 3, 4, 5])
param_1d = np.array([1, 1, 1]) # example filter in 1d

In [3]:
def zero_pad_1d(arr, n):
    p = np.array([0] * n)
    out = np.concatenate([p, arr, p])
    return out

In [5]:
print(zero_pad_1d(input_1d, 1))
print(np.pad(input_1d, 1, constant_values=0))

[0 1 2 3 4 5 0]
[0 1 2 3 4 5 0]


In [15]:
def conv_1d(arr, param):
    n_pad = len(param) // 2
    arr_padded = zero_pad_1d(arr, n_pad)
    out = np.zeros_like(arr)
    for i in range(len(arr)):
        for j in range(len(param)):
            out[i] += arr_padded[i+j] * param[j]
    return out

In [16]:
print(conv_1d(input_1d, param_1d))
print(np.convolve(input_1d, param_1d, mode="same"))

[ 3  6  9 12  9]
[ 3  6  9 12  9]


#### Backward pass

We want to compute:
- Partial derivative of the loss wrt. each element of the input, `input_1d`, to the convolution operator.
- Partial derivative of the loss wrt. each element of the filter, `param_1d`, in the convolution operator.

In a simple 1D case, where $X = [x_1, x_2, x_3, x_4, x_5]$ and $W = [w_1, w_2, w_3]$, for the output of convolve is:
$$
\begin{align}
C = \textsf{conv}(X, W) = [ 
x_0w_1 + x_1w_2 + x_2w_3 &= c_1, \\
x_1w_1 + x_2w_2 + x_3w_3 &= c_2, \\
x_2w_1 + x_3w_2 + x_4w_3 &= c_3, \\
x_3w_1 + x_4w_2 + x_5w_3 &= c_4, \\
x_4w_1 + x_5w_2 + x_6w_3 &= c_5 ]
\end{align}
$$

where $x_0$ and $x_6$ are the zero padded elements of the input. If we set the loss to be, $L = \sum_i c_i$, we can write:

$$
\begin{align}
\frac{\partial L}{\partial x_5} &= 
  \frac{\partial L}{\partial c_4}\frac{\partial c_4}{\partial x_5}
+ \frac{\partial L}{\partial c_5}\frac{\partial c_5}{\partial x_5}
+ \frac{\partial L}{\partial c_6}\frac{\partial c_4}{\partial x_6} \\
&= c_4^\textsf{grad}w_3 + c_5^\textsf{grad}w_2 + c_6^\textsf{grad}w_1 \\
\frac{\partial L}{\partial x_4} &= c_3^\textsf{grad}w_3 + c_4^\textsf{grad}w_2 + c_5^\textsf{grad}w_1 \\
\frac{\partial L}{\partial x_3} &= c_2^\textsf{grad}w_3 + c_3^\textsf{grad}w_2 + c_4^\textsf{grad}w_1
\end{align}
$$

When the loss is just a sum, we have $c_i^\textsf{grad} = 1$, and hence, for example we have $x_5^\textsf{grad} = w_2 + w_3$, note that $c_6$ is non-existant, we put it there to see the pattern.

For the gradient wrt.the weights we can write:

$$
\begin{align}
w_1^{\textsf{grad}} &= c_1^{\textsf{grad}}x_0 + c_2^{\textsf{grad}}x_1 + c_3^{\textsf{grad}}x_2 + c_4^{\textsf{grad}}x_3 + c_5^{\textsf{grad}}x_4 \\
w_2^{\textsf{grad}} &= c_1^{\textsf{grad}}x_1 + c_2^{\textsf{grad}}x_2 + c_3^{\textsf{grad}}x_3 + c_4^{\textsf{grad}}x_4 + c_5^{\textsf{grad}}x_5 \\
w_3^{\textsf{grad}} &= c_1^{\textsf{grad}}x_2 + c_2^{\textsf{grad}}x_3 + c_3^{\textsf{grad}}x_4 + c_4^{\textsf{grad}}x_5 + c_5^{\textsf{grad}}x_6
\end{align}
$$

In [61]:
# for our simple case of L=sum(c_i), the gradient of loss function
# wrt. elements of convolution output is 1 with the same shape as input
conv = conv_1d(input_1d, input_1d)
conv_1d_grad = np.ones_like(conv)

In [57]:
def input_1d_grads(input_1d, param_1d, out_grad):
    n_pad = len(param_1d) // 2
    input_grad = np.zeros_like(input_1d)
    out_grad_padded = zero_pad_1d(out_grad, n_pad)
    for i in range(len(input_1d)):
        for j in range(len(param_1d)):
            input_grad[i] += out_grad_padded[i+len(param_1d)-j-1] * param_1d[j]
    return input_grad

In [58]:
def param_1d_grads(input_1d, param_1d, out_grad):
    n_pad = len(param_1d) // 2
    input_1d_padded = zero_pad_1d(input_1d, n_pad)
    param_grad = np.zeros_like(param_1d)
    for i in range(len(input_1d)):
        for j in range(len(param_1d)):
            param_grad[j] += out_grad[i] * input_1d_padded[i+j]
    return param_grad

In [59]:
# increase one element by 1 to see the change in the loss function, 
# which for our case here, is just the sum of the convolution, 
# the change in the convolution sum for two inputs, should be equal 
# to the gradient of the respective element of that input.
input_1d_p1 = np.array([1, 2, 3, 4, 6])
param_1d_p1 = np.array([2, 1, 1])

In [62]:
print(input_1d_grads(input_1d, param_1d, conv_1d_grad))
print(np.sum(conv_1d(input_1d_p1, param_1d)) - np.sum(conv_1d(input_1d, param_1d)))

print(param_1d_grads(input_1d, param_1d, conv_1d_grad))
print(np.sum(conv_1d(input_1d, param_1d_p1)) - np.sum(conv_1d(input_1d, param_1d)))

[2 3 3 3 2]
2
[10 15 14]
10


In [71]:
# adding the batch dimension
input_1d_batch = np.array([[1, 2, 3, 4, 5], 
                           [10, 11, 2, 3, 4]])

In [53]:
def conv_1d_batch(input_1d_batch, param_1d):
    outs = [conv_1d(obs, param_1d) for obs in input_1d_batch]
    out  = np.stack(outs)
    return out

In [80]:
conv = conv_1d_batch(input_1d_batch, param_1d)
conv_1d_grad_batch = np.ones_like(conv)
print(conv)
print(conv_1d_grad_batch)

[[ 3  6  9 12  9]
 [21 23 16  9  7]]
[[1 1 1 1 1]
 [1 1 1 1 1]]


In [81]:
def input_1d_grads_batch(input_1d_batch, param_1d, out_grad):
    batch_size = len(input_1d_batch)
    input_grads = [input_1d_grads(input_1d_batch[i], param_1d, out_grad[i]) for i in range(batch_size)]
    out = np.stack(input_grads)
    return out

In [83]:
def param_1d_grads_batch(input_1d_batch, param_1d, out_grad):
    param_grad = np.zeros_like(param_1d)
    for b in range(input_1d_batch.shape[0]):
        n_pad = len(param_1d) // 2
        input_1d_padded = zero_pad_1d(input_1d_batch[b], n_pad)
        for i in range(input_1d_batch.shape[1]):
            for j in range(len(param_1d)):
                param_grad[j] += out_grad[b][i] * input_1d_padded[i+j]
    return param_grad

In [86]:
print(input_1d_grads_batch(input_1d_batch, param_1d, conv_1d_grad_batch))
print(param_1d_grads_batch(input_1d_batch, param_1d, conv_1d_grad_batch))

[[2 3 3 3 2]
 [2 3 3 3 2]]
[36 45 34]
